In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision import transforms


import torch.optim as optim

In [2]:
import pytorch_lightning as pl

##### base

In [13]:
train_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())

In [14]:
train_loader = DataLoader(train_data, batch_size=32)

In [15]:
class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28*28, 64)
        self.l2 = nn.Linear(64, 64)
        self.l3 = nn.Linear(64, 10)
        self.dropout = nn.Dropout(0.1)
    
    def forward(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        dropout = self.dropout(h2 + h1)
        logits = self.l3(dropout)
        return logits

##### Example 2: configure_optimiziers

In [138]:
_model1 = ResNet()

In [157]:
import pytorch_lightning as pl

In [158]:
_model1

ResNet(
  (l1): Linear(in_features=784, out_features=64, bias=True)
  (l2): Linear(in_features=64, out_features=64, bias=True)
  (l3): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [159]:
optimizier = optim.SGD(_model1.parameters(), lr=1e-2)

Write the `configure_optimizers` method for `LightingModule` from `_model1` and `optimizier`

In [160]:
class LitResNet(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = _model1
    
    def configure_optimizers(self):
        optimizier = optim.SGD(self.model.parameters(), lr=1e-2)
        return optimizier

In [161]:
lit_model = LitResNet()

In [162]:
lit_model

LitResNet(
  (model): ResNet(
    (l1): Linear(in_features=784, out_features=64, bias=True)
    (l2): Linear(in_features=64, out_features=64, bias=True)
    (l3): Linear(in_features=64, out_features=10, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

##### Example 3: `training_step`

In [146]:
import pytorch_lightning as pl

In [147]:
_model1

ResNet(
  (l1): Linear(in_features=784, out_features=64, bias=True)
  (l2): Linear(in_features=64, out_features=64, bias=True)
  (l3): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [11]:
loss = nn.CrossEntropyLoss()

Write the `training_step` method for `LightingModule` from `_model1`, `loss`

In [149]:
class LitResNet(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = _model1
        self.loss = nn.CrossEntropyLoss()

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1) # can be ignore
        pred = self.model(x)
        loss = self.loss(pred, y)
        
        return loss

In [150]:
lit_model = LitResNet()

In [151]:
lit_model

LitResNet(
  (model): ResNet(
    (l1): Linear(in_features=784, out_features=64, bias=True)
    (l2): Linear(in_features=64, out_features=64, bias=True)
    (l3): Linear(in_features=64, out_features=10, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (loss): CrossEntropyLoss()
)

##### Example 4: do the train

In [27]:
loss = nn.CrossEntropyLoss()

In [28]:
_model1 = ResNet()

Turn `ResNet` to `LightingModule`

In [24]:
optimizier = optim.SGD(_model1.parameters(), lr=1e-2)

In [119]:
class LitResNet(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = _model1
        self.loss = nn.CrossEntropyLoss()
    
    def configure_optimizers(self):
        optimiziers = optim.SGD(self.model.parameters(), lr=1e-2)
        return optimiziers

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1) # can be ignore
        pred = self.model(x)
        loss = self.loss(pred, y)
        
        return loss

In [120]:
lit_model = LitResNet()

In [125]:
import pytorch_lightning as pl

In [126]:
train_loader

In [124]:
lit_model

LitResNet(
  (model): ResNet(
    (l1): Linear(in_features=784, out_features=64, bias=True)
    (l2): Linear(in_features=64, out_features=64, bias=True)
    (l3): Linear(in_features=64, out_features=10, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (loss): CrossEntropyLoss()
)

`lit_model` is a lighting module and `train_loader` is the train dataloader. Do training

In [129]:
trainer = pl.Trainer(accelerator='gpu', devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [131]:
trainer.fit(lit_model, train_loader)

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /notebooks/pytorch_lighting/lightning_logs/version_3/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 55.1 K
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
55.1 K    Trainable params
0         Non-trainable params
55.1 K    Total params
0.220     Total estimated model params size (MB)


Training: 915it [00:00, ?it/s]

##### back to lesson

In [163]:
!ls lightning_logs/

version_0  version_1  version_2  version_3


In [164]:
!ls

01_basics.ipynb  Untitled.ipynb  data  lightning_logs


##### Example 5

In [6]:
from pytorch_lightning.metrics.functional import accuracy

ModuleNotFoundError: No module named 'pytorch_lightning.metrics'

In [16]:
class Linear: pass